In [ ]:
################## IMPORTS ##################

import requests
import re
import json
import datetime
import sched
import time
import random
import threading
from getpass import getpass

################## FUNCTIONS ##################

# using authentication [user,pass] to get a token from the historian
def getUserToken(auth,url):
    method = 'getUserToken'
    payload = "{\r\n  \"username\":\"%s\",\r\n  \"password\":\"%s\"\r\n}"%(auth[0],auth[1])
    headers = {'Content-Type': 'application/json','Content-Type': 'application/json'}
    resp = requests.request("POST", url+method, headers=headers, data=payload).text.encode('utf8')
    userToken = str(resp).split(r'"')[-2]
    parsed = json.loads(resp)
    if parsed.get('statusCode') == 'Good':
        return userToken
    else:
        print('\nThe %s response was not Good, it looks like this: \n\n%s'%(method,parsed))

# using the getSenderUserToken and a post to the getSessionToken endpoint to return both tokens
def getSenderTokensHardcode(url,user,pswd):
    method = 'getSessionToken'
    auth = [user,pswd]
    userToken = getUserToken(auth,url)
    payload = "{\r\n  \"userToken\":\"%s\",\r\n  \"historians\":[\"historian.domain.com\"],\r\n  \"clientId\":\"pythonWriteAPI\",\r\n  \"settings\":{\r\n  \t\"clientTimeout\":300000,\r\n  \t\"fileSize\":8,\r\n  \t\"autoCreateDatasets\": true,\r\n  \t\"autoWriteNoData\": false\r\n  }\r\n}"%userToken
    headers = {
      'Content-Type': 'application/json',
      'Content-Type': 'text/plain'
    }
    resp = requests.request("POST", url+method, headers=headers, data=payload).text.encode('utf8')
    sessToken = str(resp).split(r'"')[-2]
    parsed = json.loads(resp)
    if parsed.get('statusCode') == 'Good':
        print('I was able to get the User and Session tokens for the Write API.')
        return (userToken,sessToken)
    else:
        print('\nThe %s response was not Good, it looks like this: \n\n%s'%(method,parsed))
        return ('error','error')

# using authentication [user,pass] to get a token from the historian
def revokeSessionToken(url,utoken,stoken):
    method = 'revokeSessionToken'
    payload = "{\r\n  \"userToken\":\"%s\",\r\n  \"sessionToken\":\"%s\"\r\n}"%(utoken,stoken)
    headers = {'Content-Type': 'application/json','Content-Type': 'application/json'}
    resp = requests.request("POST", url+method, headers=headers, data=payload).text.encode('utf8')
    userToken = str(resp).split(r'"')[-2]
    parsed = json.loads(resp)
    if parsed.get('statusCode') == 'Good':
        print('\nThe Write API tokens have been revoked.')
        return 0,0
    else:
        print('\nThe %s response was not Good, it looks like this: \n\n%s'%(method,parsed))

# function for making the payload for a storeData post of a single TVQ
def storeDataMakePayload(tag,timestamp,value,utoken,stoken):
    payload = "%s\": [\r\n  \t\t[\r\n  \t\t\"%s\",\r\n  \t\t%s\r\n  \t\t]\r\n  \t]\r\n  }\r\n}"%(tag,timestamp,value)
    payload = "{\r\n  \"userToken\":\"%s\",\r\n  \"sessionToken\":\"%s\",\r\n  \"tvqs\": {\r\n  \t\""%(utoken,stoken) + payload
    return payload

def storeDataSingleTVQ(url,user,pswd,tag,timestamp,value):
    method = 'storeData'
    headers = {
      'Content-Type': 'application/json',
      'Content-Type': 'text/plain'
    }
    utoken,stoken = getSenderTokensHardcode(url,user,pswd)
    payload = storeDataMakePayload(tag,timestamp,value,utoken,stoken)
    response = requests.request("POST", url+method, headers=headers, data=payload).text.encode('utf8')
    parsed = json.loads(response)
    if parsed.get('statusCode') == 'Good':
        print(response)
    else:
        print('\nThe %s response was not Good, it looks like this: \n\n%s'%(method,parsed))
    revokeSessionToken(url,utoken,stoken)

In [ ]:
url = r'https://historian.domain.com:<port>/api/v1/'
tag = 'DummyTag.RandomValue'
timestamp = datetime.datetime.now()
value = random.random()
user = r'username'
pswd = '***' # should not hardcode passwords, only for testing purposes

In [ ]:
s = sched.scheduler(time.time, time.sleep)

def FiveSecLoop(sc):
    timestamp = datetime.datetime.now()
    value = random.random()
    url = r'https://historian.domain.com:<port>/api/v1/'
    tag = 'DummyTag.RandomValue'
    user = r'username'
    pswd = '***' # should not hardcode passwords, only for testing purposes
    storeDataSingleTVQ(url,user,pswd,tag,timestamp,value)
    sc.enter(5,1,FiveSecLoop,(sc,))

s.enter(5,1,FiveSecLoop,(s,))
s.run()

In [ ]:
FiveSecLoop(url,user,pswd,tag)